<a href="https://colab.research.google.com/github/vipratiwari/Algorithems/blob/master/sample_dask_scikit_implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
# !pip install dask scikit-learn
# !pip install dask_ml

Load the dataset

In [2]:
from sklearn.datasets import load_digits
digits = load_digits()
X, y = digits.data, digits.target

Split the dataset into training and test sets

In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Create a Dask client to enable parallel processing

In [5]:
from dask.distributed import Client
client = Client()

INFO:distributed.http.proxy:To route to workers diagnostics web server please install jupyter-server-proxy: python -m pip install jupyter-server-proxy
INFO:distributed.scheduler:State start
INFO:distributed.scheduler:  Scheduler at:     tcp://127.0.0.1:38265
INFO:distributed.scheduler:  dashboard at:            127.0.0.1:8787
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:38113'
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:34705'
INFO:distributed.scheduler:Register worker <WorkerState 'tcp://127.0.0.1:45885', name: 0, status: init, memory: 0, processing: 0>
INFO:distributed.scheduler:Starting worker compute stream, tcp://127.0.0.1:45885
INFO:distributed.core:Starting established connection to tcp://127.0.0.1:47684
INFO:distributed.scheduler:Register worker <WorkerState 'tcp://127.0.0.1:36873', name: 1, status: init, memory: 0, processing: 0>
INFO:distributed.scheduler:Starting worker compute stream, tcp://127.0.0.1:36873
INFO:distributed.core:Startin

Parallelize the data with Dask

In [6]:
import dask.array as da
X_train_dask = da.from_array(X_train, chunks=100)
y_train_dask = da.from_array(y_train, chunks=100)

Define a pipeline using Scikit-Learn's Pipeline class

In [7]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

pipeline = Pipeline(steps=[
    ('preprocessing', StandardScaler()),
    ('classifier', LogisticRegression())
])

Train the model using Dask for parallel processing

In [10]:
from dask_ml.model_selection import IncrementalSearchCV

search = IncrementalSearchCV(
    pipeline,
    param_distributions={
        'classifier__C': [0.01, 0.1, 1.0],
        'classifier__penalty': ['l1', 'l2']
    },
    scoring='accuracy',
    n_jobs=-1,
    cv=3,
    random_state=42
)

search.fit(X_train_dask, y_train_dask)

ModuleNotFoundError: ignored

Evaluate the model on the test set


In [11]:
from sklearn.metrics import accuracy_score

y_pred = search.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Test accuracy: {accuracy:.2f}')

NameError: ignored